# Setup

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy import interpolate

import os
import time

from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import FormatStrFormatter
from matplotlib.animation import FuncAnimation

import seaborn as sns
import json

In [2]:
# Enable interactive plot
%matplotlib notebook

In [3]:
from cosmo_bbh_tools import cosmo_bbh_tools as cbt

In [4]:
from astropy.cosmology import WMAP9 as cosmo
import astropy.units as u
H0 = cosmo.H(0)
t0 = 1.0/H0
t0_Myr = t0.decompose().to(u.Myr)
T_HUBBLE_MYR = t0_Myr.value
print(T_HUBBLE_MYR)

14105.485021361645


In [6]:
n_systems_all = [5e1, 5e2, 5e3, 5e4, 5e5, 5e6]
n_systems = n_systems_all[:]
print(n_systems)

zminn = 0.000100 # COMPAS MIN
zmaxx = 0.030000 # COMPAS MAX
Z_MIN_LOG = np.log10(zminn)
Z_MAX_LOG = np.log10(zmaxx)

metallicities = np.logspace(Z_MIN_LOG, Z_MAX_LOG, 10)
print(metallicities)

[50.0, 500.0, 5000.0, 50000.0, 500000.0, 5000000.0]
[0.0001     0.00018847 0.0003552  0.00066943 0.00126166 0.00237782
 0.0044814  0.00844598 0.01591789 0.03      ]


In [5]:
def get_index(n, met):
    assert met < len(metallicities), f"metallicity index cannot exceed {len(metallicities)-1}"
    return n*len(metallicities) + met

# Begin Writing useful information

In [7]:
PathData =     os.environ['WORK'] + '/cosmo_bh_grid/'

In [8]:
Z_ALL, M1_ZAMS_ALL, M2_ZAMS_ALL, M1_CO_ALL, M2_CO_ALL, \
                SMA_CO_ALL, ECC_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, DELAY_TIMES_ALL  = cbt.load_data(n_systems, metallicities, PathData)

loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.00e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.88e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_3.55e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_6.69e-04_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.26e-03_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_2.38e-03_combined.h5
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_4.48e-03_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_8.45e-03_combined.h5
No DCOs were found
loading data from /work2/08178/vkapil/frontera/cosmo_bh_grid/n_5.00e+01/met_1.59e-02_combin

In [9]:
sfm = cbt.get_sfm(n_systems, metallicities, M1_ZAMS_ALL, M2_ZAMS_ALL)
print(sfm)

Star Forming Mass in 5.00e+01 stars: 2.97e+03 M_sol 

Star Forming Mass in 5.00e+02 stars: 3.19e+04 M_sol 

Star Forming Mass in 5.00e+03 stars: 3.59e+05 M_sol 

Star Forming Mass in 5.00e+04 stars: 3.76e+06 M_sol 

Star Forming Mass in 5.00e+05 stars: 3.75e+07 M_sol 

Star Forming Mass in 5.00e+06 stars: 3.76e+08 M_sol 

[2.97380734e+03 3.19232086e+04 3.59218010e+05 3.75513954e+06
 3.75285972e+07 3.75972741e+08]


In [10]:
# Isolate BBHs
M1_BBH_ALL = cbt.mask_type(M1_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
M2_BBH_ALL = cbt.mask_type(M2_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
SMA_BBH_ALL = cbt.mask_type(SMA_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
ECC_BBH_ALL = cbt.mask_type(ECC_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
TYPE1_BBH_ALL = cbt.mask_type(TYPE1_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
TYPE2_BBH_ALL = cbt.mask_type(TYPE2_CO_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')
DELAY_TIMES_BBH_ALL  = cbt.mask_type(DELAY_TIMES_ALL, TYPE1_CO_ALL, TYPE2_CO_ALL, dco_type='bbh')

In [11]:
len(M1_BBH_ALL)

60

In [12]:
M1_BBH_METS = cbt.flatten_list_by_met(sfm, metallicities, M1_BBH_ALL)
M2_BBH_METS = cbt.flatten_list_by_met(sfm, metallicities, M2_BBH_ALL)
DELAY_TIMES_METS = cbt.flatten_list_by_met(sfm, metallicities, DELAY_TIMES_BBH_ALL)

# Save raw data

In [13]:
output_dir = "raw_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [14]:
for i in range(len(M1_BBH_ALL)):
    M1_BBH_ALL[i] = list(M1_BBH_ALL[i])

In [15]:
for i in range(len(M2_BBH_ALL)):
    M2_BBH_ALL[i] = list(M2_BBH_ALL[i])

In [16]:
for i in range(len(DELAY_TIMES_BBH_ALL)):
    DELAY_TIMES_BBH_ALL[i] = list(DELAY_TIMES_BBH_ALL[i])

In [31]:
M1_ZAMS_TOTAL = np.zeros(len(M1_ZAMS_ALL))
for i in range(len(M1_ZAMS_ALL)):
    M1_ZAMS_TOTAL[i] = np.sum(M1_ZAMS_ALL[i])
M1_ZAMS_TOTAL = list(M1_ZAMS_TOTAL)   

M2_ZAMS_TOTAL = np.zeros(len(M2_ZAMS_ALL))
for i in range(len(M2_ZAMS_ALL)):
    M2_ZAMS_TOTAL[i] = np.sum(M2_ZAMS_ALL[i])
M2_ZAMS_TOTAL = list(M2_ZAMS_TOTAL)   

In [34]:
M2_ZAMS_TOTAL

[290.3095082273579,
 290.3095082273579,
 290.3095082273579,
 290.3095082273579,
 281.2178826618714,
 291.5153175727272,
 291.5153175727272,
 291.5153175727272,
 288.7794970821875,
 288.7794970821875,
 3017.2571658193224,
 3017.2571658193224,
 3018.1616137787187,
 3024.418344742825,
 3019.9533205815433,
 3020.6746595384257,
 3001.436644397227,
 2995.795779832162,
 3015.5133034269998,
 3015.303527084696,
 33174.245693165096,
 33178.92374669037,
 33212.87561971634,
 33125.97323413142,
 33071.21449748083,
 33023.46232353951,
 33034.32315944454,
 32865.900281845985,
 33034.59291913473,
 33123.71540106221,
 348877.818347045,
 348563.1998279188,
 348467.61369533575,
 348308.468433816,
 348144.6559032222,
 347822.15981840855,
 347752.8948353584,
 347384.3895162193,
 347267.8841327211,
 346442.0677326563,
 3473504.086097972,
 3472959.698064536,
 3473252.1412879205,
 3471937.9002587865,
 3471154.585866553,
 3467613.3919066796,
 3467565.664843322,
 3465175.7658895324,
 3462994.5117307818,
 345498

In [39]:
output_dir = "raw_data"

fname = "M1_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'w') as myfile:
        json.dump(M1_ZAMS_TOTAL, myfile)
        
fname = "M2_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'w') as myfile:
        json.dump(M2_ZAMS_TOTAL, myfile)


In [18]:
import inspect

def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]


In [33]:
for data in [M1_BBH_ALL, M2_BBH_ALL, DELAY_TIMES_BBH_ALL]:
    fname = retrieve_name(data)[0]
    with open(f"{output_dir}/{fname}",'w') as myfile:
        json.dump(data, myfile)

## save M1, M2, Delay times by metallicity

In [20]:
output_dir = "bbh_by_met"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [21]:
fname = f"M1_BBH_METS"
with open(f"{output_dir}/{fname}",'w') as myfile:
    json.dump(M1_BBH_METS,myfile)
    
fname = f"M2_BBH_METS"
with open(f"{output_dir}/{fname}",'w') as myfile:
    json.dump(M2_BBH_METS,myfile)
    
fname = f"DELAY_TIMES_METS"
with open(f"{output_dir}/{fname}",'w') as myfile:
    json.dump(DELAY_TIMES_METS,myfile)

# Read the existing files

In [7]:
output_dir = "bbh_by_met"

fname = f"M1_BBH_METS"
with open(f"{output_dir}/{fname}",'r') as infile:
    M1_BBH_METS = json.load(infile)
    
fname = f"M2_BBH_METS"
with open(f"{output_dir}/{fname}",'r') as infile:
    M2_BBH_METS = json.load(infile)
    
fname = f"DELAY_TIMES_METS"
with open(f"{output_dir}/{fname}",'r') as infile:
    DELAY_TIMES_METS = json.load(infile)
    
    

output_dir = "raw_data"

fname = f"M1_BBH_ALL"
with open(f"{output_dir}/{fname}",'r') as infile:
    M1_BBH_ALL = json.load(infile)    
    
fname = f"M1_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'r') as infile:
    M1_ZAMS_TOTAL = json.load(infile)   
    
fname = f"M2_ZAMS_TOTAL"
with open(f"{output_dir}/{fname}",'r') as infile:
    M2_ZAMS_TOTAL = json.load(infile)   
    

In [8]:
sfm = cbt.get_sfm(n_systems, metallicities, M1_ZAMS_TOTAL, M2_ZAMS_TOTAL, output_loc="raw_data/sfm")
print(sfm)

Star Forming Mass in 5.00e+01 stars: 2.97e+03 M_sol 

Star Forming Mass in 5.00e+02 stars: 3.19e+04 M_sol 

Star Forming Mass in 5.00e+03 stars: 3.59e+05 M_sol 

Star Forming Mass in 5.00e+04 stars: 3.76e+06 M_sol 

Star Forming Mass in 5.00e+05 stars: 3.75e+07 M_sol 

Star Forming Mass in 5.00e+06 stars: 3.76e+08 M_sol 

[2.97380734e+03 3.19232086e+04 3.59218010e+05 3.75513954e+06
 3.75285972e+07 3.75972741e+08]
